<a href="https://colab.research.google.com/github/BarlaSowmya/AlgoTrading/blob/main/h2h_trial2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# ✅ Install required packages (only needed once)
!pip install yfinance ta gspread oauth2client scikit-learn --quiet

# ✅ Imports
import yfinance as yf
import pandas as pd
import numpy as np
import ta
import logging
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# ✅ Logging
logging.basicConfig(level=logging.INFO)

# ✅ Telegram Setup
TELEGRAM_TOKEN = "your token"  # Replace with your token
CHAT_ID = "your chat ID"  # Replace with your chat ID

def send_telegram(message):
    url = f"https://api.telegram.org/bot{TELEGRAM_TOKEN}/sendMessage"
    payload = {"chat_id": CHAT_ID, "text": message}
    requests.post(url, data=payload)


# ✅ Google Sheets Setup
SHEET_NAME = "AlgoTradingLog"
TRADE_LOG_SHEET = "P&L_Log"
TRADE_EXECUTION_SHEET = "Trade_Log"
SUMMARY_SHEET = "Summary"

# ✅ Setup Google Sheets API
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("service_account.json", scope)
sheet_client = gspread.authorize(creds)
sheet = sheet_client.open(SHEET_NAME)

# ✅ Create missing sheets and headers
sheet_names = [ws.title for ws in sheet.worksheets()]
if TRADE_LOG_SHEET not in sheet_names:
    sheet.add_worksheet(title=TRADE_LOG_SHEET, rows="100", cols="10")
log_sheet = sheet.worksheet(TRADE_LOG_SHEET)
log_sheet.clear()
log_sheet.append_row(['Symbol', 'Total Trades', 'Winning Trades', 'Win Rate (%)', 'Total Profit', 'ML Accuracy (%)'])

if TRADE_EXECUTION_SHEET not in sheet_names:
    sheet.add_worksheet(title=TRADE_EXECUTION_SHEET, rows="100", cols="10")
exec_sheet = sheet.worksheet(TRADE_EXECUTION_SHEET)
exec_sheet.clear()
exec_sheet.append_row(['Date', 'Symbol', 'Action', 'Price'])

if SUMMARY_SHEET not in sheet_names:
    sheet.add_worksheet(title=SUMMARY_SHEET, rows="100", cols="2")
summary = sheet.worksheet(SUMMARY_SHEET)
summary.clear()
summary.resize(rows=4, cols=2)
summary.update(values=[['Metric', 'Value'], ['Average Win Rate (%)', ''], ['Total Net Profit', ''], ['Avg. ML Accuracy (%)', '']], range_name='A1:B4')

# ✅ Stocks
symbols = ["INFY.NS", "TCS.NS", "RELIANCE.NS"]

# ✅ Fetch stock data
def fetch_stock_data(symbol):
    df = yf.Ticker(symbol).history(period="60d", interval="1h")
    df.reset_index(inplace=True)
    df['RSI'] = ta.momentum.RSIIndicator(df['Close'], window=14).rsi()
    df['SMA20'] = ta.trend.SMAIndicator(df['Close'], window=20).sma_indicator()
    df['SMA50'] = ta.trend.SMAIndicator(df['Close'], window=50).sma_indicator()
    return df.dropna()

# ✅ Apply strategy + log trades
def apply_strategy(df, symbol):
    df['Buy_Signal'] = False
    df['Sell_Signal'] = False
    in_position = False
    trade_sheet = sheet.worksheet(TRADE_EXECUTION_SHEET)

    for i in range(1, len(df)):
        if not in_position and df['RSI'].iloc[i] < 35 and df['SMA20'].iloc[i] > df['SMA50'].iloc[i]:
            df.at[i, 'Buy_Signal'] = True
            in_position = True
            trade_sheet.append_row([df['Datetime'].iloc[i].strftime("%Y-%m-%d %H:%M"), symbol, 'BUY', df['Close'].iloc[i]])
        elif in_position and df['RSI'].iloc[i] > 70:
            df.at[i, 'Sell_Signal'] = True
            in_position = False
            trade_sheet.append_row([df['Datetime'].iloc[i].strftime("%Y-%m-%d %H:%M"), symbol, 'SELL', df['Close'].iloc[i]])
    return df

# ✅ Backtest logic
def backtest(df):
    buys, profits = [], []
    for i in range(len(df)):
        if df['Buy_Signal'].iloc[i]:
            buys.append(df['Close'].iloc[i])
        elif df['Sell_Signal'].iloc[i] and buys:
            profit = df['Close'].iloc[i] - buys.pop(0)
            profits.append(profit)
    total = len(profits)
    win = sum([1 for p in profits if p > 0])
    return {
        "Total Trades": total,
        "Winning Trades": win,
        "Win Rate (%)": round(win / total * 100, 2) if total else 0,
        "Total Profit": round(sum(profits), 2)
    }

# ✅ ML Prediction
def ml_prediction(df):
    df['Target'] = (df['Close'].shift(-1) > df['Close']).astype(int)
    df.dropna(inplace=True)
    X = df[['RSI', 'SMA20', 'SMA50']]
    y = df['Target']
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.3)
    model = LogisticRegression()
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    return round(accuracy_score(y_test, pred) * 100, 2)

# ✅ Logging to Google Sheets
def log_to_summary():
    data = sheet.worksheet(TRADE_LOG_SHEET).get_all_records()
    valid_rows = [row for row in data if row['Symbol'] in symbols and row['Total Trades'] != '' and row['Total Trades'] != '0']
    win_rates = [float(row['Win Rate (%)']) for row in valid_rows if row['Win Rate (%)'] != '']
    ml_accs = [float(row['ML Accuracy (%)']) for row in valid_rows if row['ML Accuracy (%)'] != '']
    total_profit = sum([float(row['Total Profit']) for row in valid_rows if row['Total Profit'] != ''])

    avg_win = round(sum(win_rates) / len(win_rates), 2) if win_rates else 0
    avg_ml = round(sum(ml_accs) / len(ml_accs), 2) if ml_accs else 0

    summary = sheet.worksheet(SUMMARY_SHEET)
    summary.update(values=[
        ['Metric', 'Value'],
        ['Average Win Rate (%)', avg_win],
        ['Total Net Profit', total_profit],
        ['Avg. ML Accuracy (%)', avg_ml]
    ], range_name='A1:B4')

# ✅ Run Full Pipeline
def run_algo():
    log_sheet = sheet.worksheet(TRADE_LOG_SHEET)
    for symbol in symbols:
        logging.info(f"Processing {symbol}...")
        df = fetch_stock_data(symbol)
        df = apply_strategy(df, symbol)
        results = backtest(df)
        accuracy = ml_prediction(df)
        results['ML Accuracy (%)'] = accuracy
        log_sheet.append_row([symbol] + list(results.values()))
    log_to_summary()

# ✅ Run the Algo
run_algo()
